In [40]:
!pip install -q -U google-generativeai langchain-google-genai==3.0.0 langchain langchain-community duckduckgo-search ddgs

ERROR: Cannot install google-generativeai==0.1.0, google-generativeai==0.2.0, google-generativeai==0.2.1, google-generativeai==0.2.2, google-generativeai==0.3.0, google-generativeai==0.3.1, google-generativeai==0.3.2, google-generativeai==0.4.0, google-generativeai==0.4.1, google-generativeai==0.5.0, google-generativeai==0.5.1, google-generativeai==0.5.2, google-generativeai==0.5.3, google-generativeai==0.5.4, google-generativeai==0.6.0, google-generativeai==0.7.0, google-generativeai==0.7.1, google-generativeai==0.7.2, google-generativeai==0.8.0, google-generativeai==0.8.1, google-generativeai==0.8.2, google-generativeai==0.8.3, google-generativeai==0.8.4, google-generativeai==0.8.5 and langchain-google-genai==3.0.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


## Import required packages

In [41]:
import os
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import tool
from langchain import hub
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.prompts import PromptTemplate

## Configure model with api key

In [42]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Check if model is working fine

In [43]:
model = genai.GenerativeModel("gemini-2.5-flash")
response = model.generate_content("Hello, how are you today?")
print(f"Direct gemini response = {response.text}")

Direct gemini response = Hello! I'm doing great, thank you for asking. How are you today?


In [44]:
# Interaction via langchain
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)
langchain_response = llm.invoke("Hello, how are you today?")
print(f"langchain response = {langchain_response.content}")

langchain response = Hello! I'm doing well, thank you for asking.

How are you today?


## Create tools

In [45]:
# Tool 1: DuckDuckGo search
search = DuckDuckGoSearchRun()

@tool
def search_tool(query: str) -> str:
    """Search the query using DuckDuckGo and return the results."""
    return search.run(query)

In [70]:
# Tool 2: Arithmetic operatins
import json
@tool
def add_numbers(input_dict: str) -> float:
    """Adds two numbers num1 (float) and num2 (float) provided in a json like {"num1": float, "num2": float}."""
    js = json.loads(input_dict)
    num1 = js['num1']
    num2 = js['num2']
    return num1 + num2

@tool
def substract_operation(input_dict: str) -> float:
    """Substract two numbers a (float) and b (float) provided in a json like {"a": float, "b": float}."""
    js = json.loads(input_dict)
    a = js['a']
    b = js['b']
    return a - b

@tool
def multiply_operation(input_dict: str) -> float:
    """Multiply two numbers a (float) and b (float) provided in a json like {"a": float, "b": float}."""
    js = json.loads(input_dict)
    a = js['a']
    b = js['b']
    return a * b

@tool
def divide_numbers(input_dict: str) -> float or str:
    """Divides the first number by the second. Handles division by zero. Numbers provided in a json like {"num1": float, "num2": float}."""
    js = json.loads(input_dict)
    num1 = js['num1']
    num2 = js['num2']
    if num2 == 0:
        return "Error: Cannot divide by zero."
    return num1 / num2

In [47]:
# Custom tool
@tool
def greet_tool(name: str) -> str:
    """Greets a person by their name."""
    return f"Hello {name}!, Nice to meet you."

In [71]:
# Collect all tools
tools = [greet_tool, search_tool, add_numbers, substract_operation, multiply_operation, divide_numbers]

prompt = hub.pull("hwchase17/react")
#prompt = PromptTemplate(input_variables=["input", "agent_scratchpad"], template=prompt_template)
print(prompt)

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)
agent = create_react_agent(llm, tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

print("Agent is ready.")

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'
Agent is ready.


In [21]:
print("Let's search using egent")
search_response = agent_executor.invoke({"input": "search for current PM of UK"})
print(f"search response = {search_response['output']}")

Let's search using egent


> Entering new AgentExecutor chain...
Action: search_tool
Action Input: current PM of UKAug 7, 2023 · current，作名词译为（水，气，电）流；趋势；作形容词译为现在的；流通的。 读音：英[ˈkʌrənt]，美[ˈkɜːrənt]。 释义： adj.现在的；流通的，通用的；最近的；草写的。 n.（ … Jul 9, 2024 · current强调现在进行时或即时状态，present描述现有的状态或事物，而recent则侧重于近期发生的事件或时间段内的事情。 根据语境的不同，这些词在表达时间概念时具有不同 … current，present，recent在表示形容词性的时候发生的时间不同、词性不同、在句型中用法不同。 一、表示形容词性的时候，表示发生的时间不同 1、current指目前存在和发生的。 2、recent … Sep 2, 2024 · 华硕电脑开机出现“USB device over current status detected”提示时，可以按照以下步骤进行排查和解决： 立即断开所有USB设备。这个提示通常意味着USB接口或连接的USB … 步进电机驱动器设置成“half current”和“full current”有什么区别？ half current是在没有脉冲的情况下自动减半电流，有的是降到70%，这样就可以减少电机的发热，full current的话就是关闭这个 …The search results provided definitions and usage of the word "current" and unrelated technical issues. It did not provide the current PM of the UK. I need to refine my search query.
Action: search_tool
Action Input: Who is the current Prime Minister of the UK?The prime minister of the United Kingdom is the princip

In [73]:
print("Let's do some arithmatic operations")
addition_response = agent_executor.invoke({"input": "What is 12345 plus 678 ?", "tools": tools})
print(f"Addition response = {addition_response['output']}")

Let's do some arithmatic operations


> Entering new AgentExecutor chain...
Action: add_numbers
Action Input: {"num1": 12345, "num2": 678}13023I now know the final answer
Final Answer: 13023

> Finished chain.
Addition response = 13023


In [74]:
print("Let's do some arithmatic operations")
addition_response = agent_executor.invoke({"input": "What is the capital of Canada? How many letters are in its name? multiply by 5 to that number."})
print(f"response = {addition_response['output']}")

Let's do some arithmatic operations


> Entering new AgentExecutor chain...
Action: search_tool
Action Input: capital of Canada4 days ago - Ottawa is the capital city of Canada. It is located in the southern portion of the province of Ontario, at the confluence of the Ottawa River and the Rideau River. Ottawa borders Gatineau, Quebec, and forms the core of the Ottawa –Gatineau census metropolitan area (CMA) and the National Capital ... 2 days ago - With a population of over 41 million, it has widely varying population densities, with the majority residing in its urban areas and large areas being sparsely populated. Canada's capital is Ottawa and its three largest metropolitan areas are Toronto, Montreal, and Vancouver. 2 weeks ago - It moved to Quebec City from 1851 to 1855, then returned to Toronto from 1855 to 1859 before returning to Quebec City from 1859 to 1865. In 1857, Queen Victoria was asked to select a permanent seat of government. Her decision to make Ottawa as the permanent